In [1]:
from intent_matching import matching
import pandas as pd
import os
import json
import numpy as np
from tqdm import tqdm

In [2]:
_PATH_DATA = '../data'
_PATH_TEST_B = os.path.join(_PATH_DATA,'test_b.csv')
_PATH_TEST_A = os.path.join(_PATH_DATA,'test_a.csv')
_PATH_INTENT = os.path.join(_PATH_DATA,'intent.json')

In [3]:
dict_intent = json.load(open(_PATH_INTENT,'r'))
dict_intent.keys()

dict_keys(['overview', 'symptoms', 'main_type', 'risk_factor', 'severe', 'prevention', 'diag', 'treatment'])

In [4]:
df_test = pd.read_csv(_PATH_TEST_B)
df_test.head()

,question,intent,annotate
0,"Chào bác sĩ ạ, gần đây em có đọc được một thôn...",overview,không nên truyền máu nếu có quan hệ huyết thốn...
1,"Thưa bác sĩ, gần đây trên mạng xã hội có truyề...",overview,Vậy bác sĩ cho tôi hỏi nên ăn hoa quả trước ha...
2,"Thưa Bác sỹ, theo tôi được biết hiện tại đang ...","symptom, treatment",Vậy Bác sỹ có thể cho tôi biết các dấu hiệu củ...
3,"Thưa bác sĩ, thời tiết đang chuyển sang mù...","risk_factor, symptom","Vào mùa này, trẻ thường dễ mắc những bện..."
4,"Cho E xin tư vấn về vi rút HPV, và có thể điều...","overview, treatment","Cho E xin tư vấn về vi rút HPV, và có thể điều..."


In [5]:
def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3

In [6]:
dict_invest = {}
## init dict_invest
for k in list(dict_intent.keys()):
    dict_invest[k] = 0


In [29]:
list_sentence = list(df_test['annotate'])
list_label = list(df_test['intent'])
list_label = [item.split(',') for item in list_label]
list_matching = []
count = 0
list_investigate = []
list_precision = []
for idx, sent in tqdm(enumerate(list_sentence), total = len(list_sentence)):
#     try:
    dict_matching = matching(str(sent))
    list_res_match = dict_matching['intent']
    list_investigate.append(dict_matching)
    res_match = [item[0] for item in list_res_match]
#         print('res_match',res_match)
#         list_matching.append(res_match)

    list_labeling = [l.replace(':','') for l in list_label[idx]]
#         print('label',list_labeling)

    ## fetch intersection

    intersect = intersection(res_match,list_labeling)
    
    ## define TP,FP,FN
    TP = len(intersect)
    FP = len(list(set(res_match) - set(list_labeling)))
    if TP+FP != 0:
        precision = TP/(TP+FP)
        list_precision.append(precision)
    else:
        list_precision.append(0)
#     FN = 
#         print('intersect',intersect)
    for i in intersect:
        dict_invest[i] += 1
    if intersect:
#             print('##'*50)
        count += 1
#     except Exception as e:
#         print(str(e))

100%|██████████| 236/236 [00:07<00:00, 30.25it/s]


In [30]:
df_investigate = pd.DataFrame(list_investigate)
df_investigate

,message,intent,token
0,không nên truyền máu nếu có quan hệ huyết thốn...,"[(overview, 1.0), (prevention, 1.0)]","[thông tin, né]"
1,Vậy bác sĩ cho tôi hỏi nên ăn hoa quả trước ha...,"[(prevention, 1.0)]",[né]
2,Vậy Bác sỹ có thể cho tôi biết các dấu hiệu củ...,"[(symptoms, 1.0), (treatment, 1.0)]","[dấu hiệu, điều trị]"
3,"Vào mùa này, trẻ thường dễ mắc những bện...","[(symptoms, 1.0), (risk_factor, 1.0)]","[biểu hiện, dễ mắc]"
4,"Cho E xin tư vấn về vi rút HPV, và có thể điều...","[(treatment, 1.0)]",[điều trị]
...,...,...,...
231,Chào TS! Bệnh về tiền liệt tuyến kiểm tra có d...,"[(diag, 1.0), (treatment, 1.0)]","[kiểm tra, điều trị]"
232,Nghe nhiều người nói cần chế độ kiêng khen rất...,"[(prevention, 1.0)]",[phòng tránh]
233,Kính thưa Bác sỹ Tôi mới được các bác sỹ chẩn ...,"[(diag, 1.0)]",[chẩn đoán]
234,"Tôi bị bệnh tăng huyết áp nhiều năm nay, xin h...",[],[]


In [26]:
df_investigate['true_intent'] = list_label
df_investigate['precision'] = list_precision

ValueError: Length of values (122) does not match length of index (236)

In [24]:
# df_investigate

In [25]:
df_investigate.to_csv(os.path.join(_PATH_DATA,'test_b_investigate.csv'),header=True,index=False)

In [9]:
np.mean(list_precision)

0.5122950819672131

In [12]:
list_label_flat = sum(list_label, [])
# list_label_flat

In [13]:
dict_invest_label = {}
for item in list(dict_intent.keys()):
    dict_invest_label[item] = list_label_flat.count(item)

In [14]:
dict_invest

{'overview': 3,
 'symptoms': 1,
 'main_type': 0,
 'risk_factor': 1,
 'severe': 3,
 'prevention': 4,
 'diag': 0,
 'treatment': 61}

In [15]:
dict_invest_label

{'overview': 73,
 'symptoms': 1,
 'main_type': 2,
 'risk_factor': 3,
 'severe': 3,
 'prevention': 5,
 'diag': 1,
 'treatment': 142}

In [16]:
list_performance = []
for item in list(dict_intent.keys()):
    dict_performance = {}
    dict_performance['class'] = item
    if dict_invest_label[item] != 0:
        dict_performance['precision'] = dict_invest[item]/dict_invest_label[item]
#     else:
#         dict_performance['precision'] = 0
        dict_performance['support'] = dict_invest_label[item]
        list_performance.append(dict_performance)

In [17]:
df_invest = pd.DataFrame(list_performance)

In [18]:
df_invest

,class,precision,support
0,overview,0.041096,73
1,symptoms,1.000000,1
2,main_type,0.000000,2
3,risk_factor,0.333333,3
4,severe,1.000000,3
5,prevention,0.800000,5
6,diag,0.000000,1
7,treatment,0.429577,142


In [19]:
np.mean(list(df_invest['precision']))

0.45050083606662805

In [20]:
# count/len(df_test)